In [1]:
from task9 import *
from task8 import *
from task7 import *
from task6 import *
from copy import deepcopy
import pytest
from grammars_constants import REGEXP_CFG, GRAMMARS, GRAMMARS_DIFFERENT, CFG_EBNF
from helper import generate_rnd_start_and_final
from rpq_template_test import (
    rpq_cfpq_test,
    different_grammars_test,
    cfpq_algorithm_test,
)
from fixtures import graph
import grammars_constants
from helper import generate_rnd_graph, generate_rnd_dense_graph
import random
from constants import LABELS


TypeError: unsupported operand type(s) for |: 'type' and 'type'

In [ ]:
def test_hellings_matrix_tensor(graph, grammar):
    start_nodes, final_nodes = generate_rnd_start_and_final(graph)
    hellings = cfpq_with_hellings(
        deepcopy(grammar), deepcopy(graph), start_nodes, final_nodes
    )
    matrix = cfpq_with_matrix(
        deepcopy(grammar), deepcopy(graph), start_nodes, final_nodes
    )
    tensor = cfpq_with_tensor(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    gll = cfpq_with_gll(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    return hellings == matrix

In [ ]:
grammars: list[cfg.CFG] = grammars_constants.GRAMMARS_DIFFERENT
test_hellings_matrix_tensor(graph, grammars[0])
funcs = [generate_rnd_dense_graph, generate_rnd_graph]
fun = random.choice(funcs)
graph = fun(1, 40, LABELS)
test_hellings_matrix_tensor(graph, grammars[0])

